# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [54]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint

# Import API key
from config import Vacation_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [55]:
# Study data files
weather_data = "output_data/weather_api.csv"

# Read the weather data and the study results
weather_data = pd.read_csv("../output_data/weather_api.csv")

# Combine the data into a single dataset
weatherdf = pd.DataFrame(weather_data)
weatherdf.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Monrovia,6.3005,-10.7969,79.66,79,99,9.78,LR,1649062800
1,1,Oktyabr'skiy,54.4815,53.4710,38.14,100,100,3.44,RU,1649062800
2,2,Barrow,71.2906,-156.7887,-2.18,78,100,8.52,US,1649062800
3,3,Capitán Bado,-23.2667,-55.5333,68.63,89,37,6.69,PY,1649062800
4,4,Bambous Virieux,-20.3428,57.7575,82.65,78,75,16.28,MU,1649062800


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [56]:
# Configure gmaps with API key
gmaps.configure(api_key=Vacation_key)

In [57]:
# Store 'Lat' and 'Lng' into  locations 
locations = weatherdf[["Latitude", "Longitude"]].astype(float)

#weatherdf = weatherdf.dropna()

hum_rate = weatherdf["Humidity"].astype(float)

In [58]:
gmaps.configure(api_key="AI...")

In [59]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

# plt.savefig("vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [61]:
#Narrow down the DataFrame to find your ideal weather condition.

#A max temperature lower than 80 degrees but higher than 70.
first = weatherdf.loc[weatherdf["Max Temp"]<80,:]
second = first.loc[weatherdf["Max Temp"]>70,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Wind Speed"]<10,:]
third

#Zero cloudiness.
fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,São Filipe,14.8961,-24.4956,70.07,72,0,8.39,CV,1649062800
59,59,Flinders,-34.5833,150.8552,70.38,60,0,6.82,AU,1649062800
103,103,Port Augusta,-32.5000,137.7667,71.53,36,0,5.84,AU,1649062800
171,171,Ürümqi,43.8010,87.6005,72.55,12,0,5.30,CN,1649062800
189,189,San Ignacio,17.1561,-89.0714,75.34,94,0,2.17,BZ,1649062800


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [62]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Latitude']
hotel_df['Lng'] = fifth['Longitude']
#hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng
43,São Filipe,CV,14.8961,-24.4956
59,Flinders,AU,-34.5833,150.8552
103,Port Augusta,AU,-32.5000,137.7667
171,Ürümqi,CN,43.8010,87.6005
189,San Ignacio,BZ,17.1561,-89.0714


In [63]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={Vacation_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Null")
        
    print("***********")

Retrieving Results for Index 43: São Filipe.
Closest hotel to São Filipe at 14.8961 , -24.4956 is Hotel Ocean View & Restaurante Seafood.
***********
Retrieving Results for Index 59: Flinders.
Closest hotel to Flinders at -34.5833 , 150.8552 is Pelican Pre-School.
***********
Retrieving Results for Index 103: Port Augusta.
Closest hotel to Port Augusta at -32.5 , 137.7667 is Comfort Inn & Suites Augusta Westside.
***********
Retrieving Results for Index 171: Ürümqi.
Closest hotel to Ürümqi at 43.801 , 87.6005 is 新天国际.
***********
Retrieving Results for Index 189: San Ignacio.
Closest hotel to San Ignacio at 17.1561 , -89.0714 is Midas Belize.
***********
Retrieving Results for Index 468: Turaif.
Closest hotel to Turaif at 31.6725 , 38.6637 is Al Rajhi ATM.
***********
Retrieving Results for Index 475: Aksu.
Closest hotel to Aksu at 41.1231 , 80.2644 is Xinwei Garden.
***********
Retrieving Results for Index 492: Broken Hill.
Closest hotel to Broken Hill at -31.95 , 141.4333 is The Gran

In [64]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
43,São Filipe,CV,14.8961,-24.4956,Hotel Ocean View & Restaurante Seafood
59,Flinders,AU,-34.5833,150.8552,Pelican Pre-School
103,Port Augusta,AU,-32.5000,137.7667,Comfort Inn & Suites Augusta Westside
171,Ürümqi,CN,43.8010,87.6005,新天国际
189,San Ignacio,BZ,17.1561,-89.0714,Midas Belize


In [65]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [66]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
heat_layer = gmaps.heatmap_layer(locations, weights=hotel_df["Humidity"],
                                max_intensity=200, point_radius=30)
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig

# Display figure 
# Heat map could not generate due to API issue


KeyError: 'Humidity'